In [292]:
import pandas as pd
import math
import numpy as np
from itertools import combinations

In [293]:
df = pd.read_csv('/home/gemmara/Downloads/supermarket.zip', compression='zip')
df

,Customer Id,Date,Product
0,JKML5ZWXP0XQ,2020-07-20,tropical fruit
1,YOGWQIUPDAJ7,2020-04-30,whole milk
2,Q3NL22281AF6,2020-09-18,pip fruit
3,C59GRTLMZQAU,2020-12-11,other vegetables
4,WUSF8LTEQZ5L,2020-01-02,whole milk
...,...,...,...
32705,PM1BQU9H9S2Q,2019-06-05,oil
32706,6AWIAP3ZV551,2019-08-10,sliced cheese
32707,LRWVFLSC8X5B,2019-02-23,candy
32708,8KYRMELYKNIH,2019-04-16,cake bar


### تعداد محصولات یکتا در کل دادگان، چندتاست؟

In [294]:
df.Product.unique().size

137

### میزان فروش به ازای هر روز به صورت میانگین چند است (تا دو رقم بعد از اعشار)؟


In [295]:
math.floor(df.groupby('Date').apply(lambda group: group.shape[0]).mean() * 100) / 100
# round(df.shape[0] / df['Date'].nunique())

44.93

### چهار محصولی که کمتر از بقیه در سبد مشتریان قرار گرفته‌اند، کدامند (ترتیب مهم نیست)؟

In [296]:
df.groupby(['Product']).apply(lambda group: group.shape[0]).sort_values()[:4]

Product
kitchen utensil          1
preservation products    1
baby cosmetics           3
bags                     4
dtype: int64

### پنج مشتری‌ای که در سال ۲۰۲۰، بیشترین تعداد "سبد" را داشته‌اند، کدامند (ترتیب مهم نیست)؟

In [297]:
# df[df['Date'].str[:4] == '2020'].groupby(['Customer Id', 'Date']).size()
list(df[(df['Date'].str[:4] == '2020')][['Customer Id', 'Date']].drop_duplicates()['Customer Id'].value_counts().sort_values(ascending=False)[:5].index)

['M722MPITBY07',
 '0ZRF32AJ06BC',
 'QBUS9NZY39BE',
 'E7FD9BU4OK0F',
 'M2DT2Z00NQCH']

### کدام روز هفته، بیشترین تعداد محصول فروش رفته‌است؟

In [298]:
df['Day_of_Week'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.day_name()
df['Day_of_Week'].max()

'Wednesday'

### پنج محصولی که بیشترین support را دارند به ترتیب نزولی کدامند؟


In [299]:
# check if there is duplicated products in each bucket
df[df.duplicated()]

,Customer Id,Date,Product,Day_of_Week


In [300]:
buckets_counts = df.groupby(['Customer Id', 'Date']).size().shape[0]
each_p_count = df.groupby('Product')['Customer Id'].count().sort_values(ascending=False).rename('support') / buckets_counts
list(each_p_count[:5].keys())

df

,Customer Id,Date,Product,Day_of_Week
0,JKML5ZWXP0XQ,2020-07-20,tropical fruit,Monday
1,YOGWQIUPDAJ7,2020-04-30,whole milk,Thursday
2,Q3NL22281AF6,2020-09-18,pip fruit,Friday
3,C59GRTLMZQAU,2020-12-11,other vegetables,Friday
4,WUSF8LTEQZ5L,2020-01-02,whole milk,Thursday
...,...,...,...,...
32705,PM1BQU9H9S2Q,2019-06-05,oil,Wednesday
32706,6AWIAP3ZV551,2019-08-10,sliced cheese,Saturday
32707,LRWVFLSC8X5B,2019-02-23,candy,Saturday
32708,8KYRMELYKNIH,2019-04-16,cake bar,Tuesday


In [301]:
df = df.merge(each_p_count, left_on='Product', right_index=True, how='left')
df

,Customer Id,Date,Product,Day_of_Week,support
0,JKML5ZWXP0XQ,2020-07-20,tropical fruit,Monday,0.068732
1,YOGWQIUPDAJ7,2020-04-30,whole milk,Thursday,0.160171
2,Q3NL22281AF6,2020-09-18,pip fruit,Friday,0.049753
3,C59GRTLMZQAU,2020-12-11,other vegetables,Friday,0.123839
4,WUSF8LTEQZ5L,2020-01-02,whole milk,Thursday,0.160171
...,...,...,...,...,...
32705,PM1BQU9H9S2Q,2019-06-05,oil,Wednesday,0.015116
32706,6AWIAP3ZV551,2019-08-10,sliced cheese,Saturday,0.014234
32707,LRWVFLSC8X5B,2019-02-23,candy,Saturday,0.014573
32708,8KYRMELYKNIH,2019-04-16,cake bar,Tuesday,0.006236


### از بین مواردی که support آن‌ها، حداقل ۰.۰۱ می‌باشد. دو الگویی که بیشترین confidence را دارند، به ترتیب نزولی کدامند؟


In [302]:
support_gt_1_percent = df[df['support'] >= 0.01]
support_gt_1_percent

,Customer Id,Date,Product,Day_of_Week,support
0,JKML5ZWXP0XQ,2020-07-20,tropical fruit,Monday,0.068732
1,YOGWQIUPDAJ7,2020-04-30,whole milk,Thursday,0.160171
2,Q3NL22281AF6,2020-09-18,pip fruit,Friday,0.049753
3,C59GRTLMZQAU,2020-12-11,other vegetables,Friday,0.123839
4,WUSF8LTEQZ5L,2020-01-02,whole milk,Thursday,0.160171
...,...,...,...,...,...
32704,06EKB4N5JE0M,2019-06-18,long life bakery product,Tuesday,0.018166
32705,PM1BQU9H9S2Q,2019-06-05,oil,Wednesday,0.015116
32706,6AWIAP3ZV551,2019-08-10,sliced cheese,Saturday,0.014234
32707,LRWVFLSC8X5B,2019-02-23,candy,Saturday,0.014573


In [303]:
support_df = df.groupby('Product')['Customer Id'].nunique().reset_index()
support_df.columns = ['Product', 'Support']
support_df

,Product,Support
0,Instant food products,60
1,UHT-milk,306
2,abrasive cleaner,22
3,artif. sweetener,29
4,baby cosmetics,3
...,...,...
132,whipped/sour cream,603
133,white bread,346
134,whole milk,1786
135,yogurt,1103


In [304]:
products = df.Product.unique()
conf_df  = pivot_table = pd.DataFrame(0, index=products, columns=products)

In [305]:
def create_pairs(group):
    pairs_list = list(combinations(group['Product'], 2))
    for pair in pairs_list:
        conf_df.loc[pair[0], pair[1]] += 1
        conf_df.loc[pair[1], pair[0]] += 1

    

df.groupby(['Customer Id', 'Date']).apply(create_pairs)
conf_df = conf_df.astype('float64')
conf_df

,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,chicken,butter,...,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
tropical fruit,0.0,123.0,31.0,94.0,91.0,8.0,43.0,17.0,21.0,23.0,...,2.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
whole milk,123.0,0.0,99.0,222.0,209.0,15.0,107.0,70.0,51.0,70.0,...,9.0,6.0,1.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0
pip fruit,31.0,99.0,0.0,74.0,74.0,7.0,22.0,13.0,16.0,17.0,...,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
other vegetables,94.0,222.0,74.0,0.0,158.0,15.0,72.0,42.0,33.0,43.0,...,4.0,4.0,1.0,0.0,0.0,3.0,2.0,1.0,0.0,0.0
rolls/buns,91.0,209.0,74.0,158.0,0.0,10.0,70.0,24.0,43.0,43.0,...,7.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pudding powder,1.0,2.0,2.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ready soups,1.0,4.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
make up remover,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
toilet cleaner,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [306]:
def calculate_confidens():
    for row_label in conf_df.index:
        for col_label in conf_df.columns:
            value = conf_df.at[row_label, col_label] 
            support_x = support_df[support_df['Product'] == row_label]['Support'].values[0]
            confidens = value / support_x
            conf_df.at[row_label, col_label] = confidens
        
calculate_confidens()

In [307]:
conf_df

,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,chicken,butter,...,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
tropical fruit,0.000000,0.135016,0.034029,0.103183,0.099890,0.008782,0.047201,0.018661,0.023052,0.025247,...,0.002195,0.004391,0.001098,0.00000,0.0,0.001098,0.001098,0.000000,0.000000,0.0
whole milk,0.068869,0.000000,0.055431,0.124300,0.117021,0.008399,0.059910,0.039194,0.028555,0.039194,...,0.005039,0.003359,0.000560,0.00056,0.0,0.001120,0.002240,0.000000,0.000000,0.0
pip fruit,0.046617,0.148872,0.000000,0.111278,0.111278,0.010526,0.033083,0.019549,0.024060,0.025564,...,0.001504,0.001504,0.001504,0.00000,0.0,0.003008,0.000000,0.000000,0.000000,0.0
other vegetables,0.064033,0.151226,0.050409,0.000000,0.107629,0.010218,0.049046,0.028610,0.022480,0.029292,...,0.002725,0.002725,0.000681,0.00000,0.0,0.002044,0.001362,0.000681,0.000000,0.0
rolls/buns,0.066764,0.153338,0.054292,0.115921,0.000000,0.007337,0.051357,0.017608,0.031548,0.031548,...,0.005136,0.003668,0.000000,0.00000,0.0,0.000000,0.000000,0.000734,0.000734,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pudding powder,0.062500,0.125000,0.125000,0.187500,0.000000,0.062500,0.062500,0.000000,0.000000,0.062500,...,0.062500,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
ready soups,0.066667,0.266667,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
make up remover,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
toilet cleaner,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [308]:
conf_df.stack().nlargest(5)

kitchen utensil        rolls/buns       1.0
                       bottled water    1.0
                       pasta            1.0
preservation products  soups            1.0
bags                   yogurt           0.5
dtype: float64